# Model WO factor


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier , GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold , cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score ,mean_squared_error
from sklearn.preprocessing import LabelEncoder
import os

import pandas as pd
import numpy as np


I need python version 3.9.17 in order to run 
import pymssql

In [2]:
from Preprocessing_Application import DataPreprocessor

server = 'reporting-db.nystartfinans.net'
database = 'reporting-db'
username = 'Andreas'
password = 'nCq8Sg@1lYnd(E'


path = "/Users/andreasnilsson/Desktop/PD 2/DATA/Gross WO & Ever90 & RunOff Monthly.sql"

preprocessor = DataPreprocessor(server, database, username, password)

df = preprocessor.fetch_data_from_sql(path)



/Users/andreasnilsson/Library/CloudStorage/OneDrive-Nstart/Skrivbordet/Repository Homes/Data-PD-Modelling/Codes/Preprocessing_Application.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


## FEATURES

### CurrentAmountAdjusted is only for healthy accounts - e.g Run off NGO 

### MaxAmountAdjusted from all accounts, if we divide WO_Amount e.g 



In [12]:
df

,AccountNumber,mob,DisbursedDate,SnapshotDate,Amount,L1_Amount,Ever90,CurrentAmount,IsOpen,SendToCollection,WO_flag,maxAmount,MaxAmountAdjusted,CurrentAmountAdjusted,WO_Amount,run_off,WO_Amount2,Row,YearMonth
0,5020169,0,2018-05-29,2018-05-31,129579.00,NaN,0,129579.00,1,1900-01-01,0,129579.00,129579.00,129579.00,NaN,1.000000,0.0,1,2018-05
1,5020169,1,2018-05-29,2018-06-30,129308.00,129579.00,0,129308.00,1,1900-01-01,0,129579.00,129579.00,129308.00,NaN,0.997909,0.0,1,2018-05
2,5020169,2,2018-05-29,2018-07-31,129022.93,129308.00,0,129022.93,1,1900-01-01,0,129579.00,129579.00,129022.93,NaN,0.995709,0.0,1,2018-05
3,5020169,3,2018-05-29,2018-08-31,0.00,129022.93,0,0.00,0,1900-01-01,0,129579.00,129579.00,0.00,NaN,0.000000,0.0,1,2018-05
4,5020169,4,2018-05-29,2018-09-30,0.00,0.00,0,0.00,0,1900-01-01,0,129579.00,129579.00,0.00,NaN,0.000000,0.0,1,2018-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396850,7972672,0,2023-10-30,2023-10-31,62979.71,NaN,0,62979.71,1,1900-01-01,0,62979.71,62979.71,62979.71,NaN,1.000000,0.0,1,2023-10
396851,7972862,0,2023-10-30,2023-10-31,162612.00,NaN,0,162612.00,1,1900-01-01,0,162612.00,162612.00,162612.00,NaN,1.000000,0.0,1,2023-10
396852,7973571,0,2023-10-31,2023-10-31,320000.00,NaN,0,320000.00,1,1900-01-01,0,320000.00,320000.00,320000.00,NaN,1.000000,0.0,1,2023-10
396853,7973704,0,2023-10-31,2023-10-31,317000.00,NaN,0,317000.00,1,1900-01-01,0,317000.00,317000.00,317000.00,NaN,1.000000,0.0,1,2023-10


In [4]:
df['Row'] = 1 
df['DisbursedDate'] = pd.to_datetime(df['DisbursedDate'])


# Combine Year and Month into a 'YearMonth' column for grouping
df['YearMonth'] = df['DisbursedDate'].dt.to_period('M')

# Now group by 'mob', 'YearMonth' and then sum 'Row', 'CurrentAmount', 'Ever90', and 'WO_flag'
vintage_data = df.groupby(['mob', 'YearMonth']).agg({'Row': 'sum', 'CurrentAmount': 'sum', 'Ever90': 'sum', 'WO_flag': 'sum'}).reset_index()

In [6]:

vintage_data

,mob,YearMonth,Row,CurrentAmount,Ever90,WO_flag
0,-1,2021-04,1,300000.00,0,0
1,0,2018-01,79,15281123.88,0,0
2,0,2018-02,86,16730558.44,0,0
3,0,2018-03,74,12767006.73,0,0
4,0,2018-04,70,11209535.25,0,0
...,...,...,...,...,...,...
2481,67,2018-02,87,3145216.41,18,15
2482,67,2018-03,74,1495358.77,10,10
2483,68,2018-01,79,2012131.26,12,10
2484,68,2018-02,87,3101934.87,19,15


In [5]:
# vintage_data[vintage_data.Year == 2018].sort_values(by = ['Year','Month','mob' ]).head(60)

In [14]:
# ## Controll Ever90 

# see2018_1_12 =  df[(df.Year == 2018) & ( df.Ever90 == 1  ) &  ( df.mob == 24  )  ]

# print(len(see2018_1_12))

# see2018_1_136 = df[(df.Year == 2018) & ( df.Ever90 == 1  ) &  ( df.mob == 56  )  ]

# print(len(pd.merge(see2018_1_12, see2018_1_136, on='AccountNumber', how='inner')))

# ## Controll WO_flag 

# see2018_1_12 =  df[(df.Year == 2018) & ( df.WO_flag == 1  ) &  ( df.mob == 12  )  ]

# print(len(see2018_1_12))

# see2018_1_136 = df[(df.Year == 2018) & ( df.WO_flag == 1  ) &  ( df.mob == 46  )  ]

# intersection = pd.merge(see2018_1_12, see2018_1_136, on='AccountNumber', how='inner')

# print(len(intersection))

In [7]:
run_off = df.groupby(['mob', 'YearMonth']).agg({'run_off': 'mean'}).reset_index()
run_off

,mob,YearMonth,run_off
0,-1,2021-04,1.000000
1,0,2018-01,0.872112
2,0,2018-02,0.825472
3,0,2018-03,0.851328
4,0,2018-04,0.828409
...,...,...,...
2481,67,2018-02,0.063811
2482,67,2018-03,0.032508
2483,68,2018-01,0.036363
2484,68,2018-02,0.058105


In [8]:
all =  pd.merge(  vintage_data , run_off , on=['mob', 'YearMonth'], how='inner') 


In [9]:
all['Ever90_%'] = all.Ever90/ all.Row 
all['WO_flag_%'] = all.WO_flag/ all.Row 

In [10]:
all.to_excel("WO_Monthly.xlsx", index=False)

In [11]:
all

,mob,YearMonth,Row,CurrentAmount,Ever90,WO_flag,run_off,Ever90_%,WO_flag_%
0,-1,2021-04,1,300000.00,0,0,1.000000,0.000000,0.000000
1,0,2018-01,79,15281123.88,0,0,0.872112,0.000000,0.000000
2,0,2018-02,86,16730558.44,0,0,0.825472,0.000000,0.000000
3,0,2018-03,74,12767006.73,0,0,0.851328,0.000000,0.000000
4,0,2018-04,70,11209535.25,0,0,0.828409,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
2481,67,2018-02,87,3145216.41,18,15,0.063811,0.206897,0.172414
2482,67,2018-03,74,1495358.77,10,10,0.032508,0.135135,0.135135
2483,68,2018-01,79,2012131.26,12,10,0.036363,0.151899,0.126582
2484,68,2018-02,87,3101934.87,19,15,0.058105,0.218391,0.172414


In [61]:
see2018_1_12[~see2018_1_12['AccountNumber'].isin(intersection['AccountNumber'])]

,AccountNumber,mob,DisbursedDate,SnapshotDate,Ever90,CurrentAmount,SendToCollection,WO_flag,run_off,Row,Year
12475,5208681,12,2018-03-06,2019-03-31,1,83247.35,2019-03-05,1,0.931387,1,2018
24735,5266200,12,2018-05-21,2019-05-31,1,183716.71,2019-05-03,1,0.993063,1,2018
62856,5453592,12,2018-11-07,2019-11-30,1,120624.01,2019-10-04,1,0.911807,1,2018
64918,5467535,12,2018-11-16,2019-11-30,1,96545.79,2019-11-04,1,0.965458,1,2018
70484,5502471,12,2018-12-07,2019-12-31,1,113506.60,2019-12-16,1,0.988055,1,2018


In [64]:
df[df.AccountNumber =='5208681'].head(60)

,AccountNumber,mob,DisbursedDate,SnapshotDate,Ever90,CurrentAmount,SendToCollection,WO_flag,run_off,Row,Year
12463,5208681,0,2018-03-06,2018-03-31,0,89380.00,2019-03-05,0,1.000000,1,2018
12464,5208681,1,2018-03-06,2018-04-30,0,88475.65,2019-03-05,0,0.989882,1,2018
12465,5208681,2,2018-03-06,2018-05-31,0,87556.30,2019-03-05,0,0.979596,1,2018
12466,5208681,3,2018-03-06,2018-06-30,0,86621.71,2019-03-05,0,0.969140,1,2018
12467,5208681,4,2018-03-06,2018-07-31,0,86621.71,2019-03-05,0,0.969140,1,2018
12468,5208681,5,2018-03-06,2018-08-31,0,85671.62,2019-03-05,0,0.958510,1,2018
12469,5208681,6,2018-03-06,2018-09-30,0,85671.62,2019-03-05,0,0.958510,1,2018
12470,5208681,7,2018-03-06,2018-10-31,0,85671.62,2019-03-05,0,0.958510,1,2018
12471,5208681,8,2018-03-06,2018-11-30,1,83247.35,2019-03-05,0,0.931387,1,2018
12472,5208681,9,2018-03-06,2018-12-31,1,83247.35,2019-03-05,0,0.931387,1,2018
